In [1]:
import pandas as pd
import ast
import os

file_path = r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Raw_Data\chennai_cars.xlsx"
df = pd.read_excel(file_path)

# Convert JSON strings to dictionaries in 'new_car_detail'
df['new_car_detail'] = df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extract specific details from 'new_car_detail'
df['fuel_type'] = df['new_car_detail'].apply(lambda x: x.get('ft'))
df['body_type'] = df['new_car_detail'].apply(lambda x: x.get('bt'))
df['kilometers_driven'] = df['new_car_detail'].apply(lambda x: x.get('km'))
df['transmission'] = df['new_car_detail'].apply(lambda x: x.get('transmission'))
df['owner'] = df['new_car_detail'].apply(lambda x: x.get('owner'))
df['oem'] = df['new_car_detail'].apply(lambda x: x.get('oem'))
df['model'] = df['new_car_detail'].apply(lambda x: x.get('model'))
df['year'] = df['new_car_detail'].apply(lambda x: x.get('modelYear'))
df['variant'] = df['new_car_detail'].apply(lambda x: x.get('variantName'))
df['price'] = df['new_car_detail'].apply(lambda x: x.get('price'))

# Convert JSON strings to dictionaries in 'new_car_overview'
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extract specific details from 'new_car_overview'
def extract_overview_details(data):
    details = {}
    for item in data.get('top', []):
        details[item['key']] = item['value']
    return details

# Apply the extraction function
overview_details = df['new_car_overview'].apply(extract_overview_details).apply(pd.Series)

# Assign extracted details to new columns
df['registration_year'] = overview_details.get('Registration Year')
df['insurance_validity'] = overview_details.get('Insurance Validity')
df['fuel_type_overview'] = overview_details.get('Fuel Type')
df['seats'] = overview_details.get('Seats')
df['kms_driven'] = overview_details.get('Kms Driven')
df['rto'] = overview_details.get('RTO')

# Convert JSON strings to dictionaries in 'new_car_feature'
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Function to extract features based on heading and subHeading
def extract_features(data, heading, subheading=None):
    for item in data['data']:
        if item['heading'] == heading and (subheading is None or item['subHeading'] == subheading):
            return ', '.join([feature['value'] for feature in item['list']])
    return None

# Extract specific features under different headings and subheadings
df['comfort_convenience'] = df['new_car_feature'].apply(lambda x: extract_features(x, 'Comfort & Convenience', 'Comfort'))
df['interior_features'] = df['new_car_feature'].apply(lambda x: extract_features(x, 'Interior', 'Interior'))
df['exterior_features'] = df['new_car_feature'].apply(lambda x: extract_features(x, 'Exterior', 'Exterior'))
df['safety_features'] = df['new_car_feature'].apply(lambda x: extract_features(x, 'Safety', 'Safety'))
df['entertainment_features'] = df['new_car_feature'].apply(lambda x: extract_features(x, 'Entertainment & Communication', 'Entertainment'))

# Convert JSON strings to dictionaries in 'new_car_specs'
df['new_car_specs'] = df['new_car_specs'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Function to extract specifications based on heading and subHeading
def extract_specifications(data, heading, subheading=None):
    for item in data['data']:
        if item['heading'] == heading and (subheading is None or item['subHeading'] == subheading):
            return {spec['key']: spec['value'] for spec in item['list']}
    return {}

# Extract specifications under different headings and subheadings
specifications = df['new_car_specs'].apply(lambda x: extract_specifications(x, 'Engine and Transmission', 'Engine'))
specifications_df = pd.DataFrame(specifications.tolist())

# Debugging: Check the keys in specifications_df
print("Available keys in specifications_df:", specifications_df.columns.tolist())

# Assign extracted details to new columns with error handling
df['mileage'] = specifications_df.get('Mileage', None)
df['engine'] = specifications_df.get('Engine', None)
df['max_power'] = specifications_df.get('Max Power', None)
df['torque'] = specifications_df.get('Torque', None)
df['wheel_size'] = specifications_df.get('Wheel Size', None)

# Separate max power into bhp and rpm, handling potential missing values
df[['bhp', 'rpm']] = df['max_power'].str.extract(r'(?P<bhp>\d+\.\d+)bhp@(?P<rpm>\d+)rpm')

# Consolidate all extracted data into a new column
df['consolidated_data'] = df.apply(lambda row: {
    'fuel_type': row['fuel_type'],
    'body_type': row['body_type'],
    'kilometers_driven': row['kilometers_driven'],
    'transmission': row['transmission'],
    'owner': row['owner'],
    'oem': row['oem'],
    'model': row['model'],
    'year': row['year'],
    'variant': row['variant'],
    'price': row['price'],
    'registration_year': row['registration_year'],
    'insurance_validity': row['insurance_validity'],
    'fuel_type_overview': row['fuel_type_overview'],
    'seats': row['seats'],
    'kms_driven': row['kms_driven'],
    'rto': row['rto'],
    'comfort_convenience': row['comfort_convenience'],
    'interior_features': row['interior_features'],
    'exterior_features': row['exterior_features'],
    'safety_features': row['safety_features'],
    'entertainment_features': row['entertainment_features'],
    'mileage': row['mileage'],
    'engine': row['engine'],
    'torque': row['torque'],
    'wheel_size': row['wheel_size'],
    'bhp': row['bhp'],
    'rpm': row['rpm'],
}, axis=1)

# Drop the original columns
df = df.drop(columns=['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs'])

# Save the structured data
output_file_path = r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Processed_Data\Chennai_structured.xlsx"
# Ensure the output directory exists
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

df.to_excel(output_file_path, index=False)


Available keys in specifications_df: ['Color', 'Engine Type', 'Displacement', 'Max Power', 'Max Torque', 'No of Cylinder', 'Values per Cylinder', 'Fuel Suppy System', 'Turbo Charger', 'Value Configuration', 'Compression Ratio', 'Super Charger', 'BoreX Stroke']


In [2]:
import pandas as pd

# Load your DataFrame (make sure to adjust the path as needed)
df = pd.read_excel(r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Processed_Data\Chennai_structured.xlsx")

# Preprocessing
df['kilometers_driven'] = df['kilometers_driven'].str.replace(',', '').astype(float)  # Remove commas and convert to float
df['owner'] = df['owner'].str.replace(' Owner', '')  # Remove the word "Owner"
df['registration_year'] = df['registration_year'].str.extract(r'(\d+)')[0]  # Keep only numeric year
df['seats'] = df['seats'].str.replace(' Seats', '')  # Remove the word "Seats"

# Fill NaN values with 'N/A' only in object-type columns
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna('N/A')  # Reassign filled values back to the DataFrame

# Save the preprocessed data to CSV
output_file_path_csv = r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Processed_Data\Chennai_preprocessed.csv"
df.to_csv(output_file_path_csv, index=False)

# Display the preprocessed DataFrame
print(df)


                                              car_links     City fuel_type  \
0     https://www.cardekho.com/used-car-details/used...  Chennai    Petrol   
1     https://www.cardekho.com/buy-used-car-details/...  Chennai    Petrol   
2     https://www.cardekho.com/used-car-details/used...  Chennai    Petrol   
3     https://www.cardekho.com/buy-used-car-details/...  Chennai    Petrol   
4     https://www.cardekho.com/used-car-details/used...  Chennai    Petrol   
...                                                 ...      ...       ...   
1414  https://www.cardekho.com/used-car-details/used...  Chennai    Petrol   
1415  https://www.cardekho.com/used-car-details/used...  Chennai    Diesel   
1416  https://www.cardekho.com/used-car-details/used...  Chennai    Petrol   
1417  https://www.cardekho.com/used-car-details/used...  Chennai    Diesel   
1418  https://www.cardekho.com/used-car-details/used...  Chennai    Diesel   

      body_type  kilometers_driven transmission owner         o